# Derivation of Euler 321 equations
T. Fitzgerald

In [1]:
using Pkg
Pkg.activate(".")
using SymPy
using LinearAlgebra

  Activating project at `c:\Users\tim\Documents\2025FallExamples\11-12`


## Define the rotations and build $C$

In [2]:
ψ, θ, ϕ = symbols("psi theta phi", real=true)

M1(θ) = [1 0 0;
    0 cos(θ) sin(θ);
    0 -sin(θ) cos(θ)]

M2(θ) = [cos(θ) 0 -sin(θ);
    0 1 0;
    sin(θ) 0 cos(θ)]

M3(θ) = [cos(θ) sin(θ) 0;
    -sin(θ) cos(θ) 0
    0 0 1]

M3 (generic function with 1 method)

In [3]:
C = M1(ϕ)*M2(θ)*M3(ψ)

3×3 Matrix{Sym{PyCall.PyObject}}:
                              cos(psi)*cos(theta)  …          -sin(theta)
 sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)     sin(phi)*cos(theta)
 sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)     cos(phi)*cos(theta)

Additionally, we can go from $C \to (\psi,\theta,\phi)$ by carefully looking at some of the entries in $C$

In [4]:
C[1,2]/C[1,1] |> simplify

tan(psi)

In [5]:
-C[1,3]

sin(theta)

In [6]:
C[2,3]/C[3,3] |> simplify

tan(phi)

## Angular Velocity

$${}^{B/N}\vec{\omega} = \dot{\psi}\hat{n}_3 + \dot{\theta}\hat{b}'_2 + \dot{\phi} \hat{b}_1$$

where $\hat{b}'_2$ is the direction of $\hat{b}_2$ before the last rotation.  Rearranging the equations into a single matrix we can build

$${}^{B/N}\vec{\omega} = \begin{bmatrix} \hat{n}_3 & \hat{b}'_2 & \hat{b}_1 \end{bmatrix}\begin{Bmatrix} \dot\psi \\ \dot\theta \\ \dot\phi \end{Bmatrix}$$

Putting everything into the same $\hat{b}$ frame we can work to build the matrix relation

$$ \begin{Bmatrix} \dot\psi \\ \dot\theta \\ \dot\phi \end{Bmatrix} = B \vec{\omega} $$

This is a differential equation that relates measured angular velocities as functions of time in the body frame and the rates of the Euler angles.

We build $B^{-1}$ by column and then invert it to get $B$

$$ B^{-1} = \begin{bmatrix} \hat{n}_3 & \hat{b}'_2 & \hat{b}_1 \end{bmatrix}$$

In [7]:
simplify.( C*[0;0;1] )

3-element Vector{Sym{PyCall.PyObject}}:
         -sin(theta)
 sin(phi)*cos(theta)
 cos(phi)*cos(theta)

In [8]:
M1(ϕ)*[0;1;0]

3-element Vector{Sym{PyCall.PyObject}}:
         0
  cos(phi)
 -sin(phi)

In [9]:
[1;0;0]

3-element Vector{Int64}:
 1
 0
 0

In [10]:
Binv = hcat(C * [0; 0; 1], M1(ϕ) * [0; 1; 0], [1; 0; 0])

3×3 Matrix{Sym{PyCall.PyObject}}:
         -sin(theta)          0  1
 sin(phi)*cos(theta)   cos(phi)  0
 cos(phi)*cos(theta)  -sin(phi)  0

In [11]:
B = simplify.( inv(Binv) )

3×3 Matrix{Sym{PyCall.PyObject}}:
 0  sin(phi)/cos(theta)  cos(phi)/cos(theta)
 0             cos(phi)            -sin(phi)
 1  sin(phi)*tan(theta)  cos(phi)*tan(theta)

sympy is doing a strange job of simplifying here and this is not a typical representation.  But is it correct and equivalent to the way it is usually reported.

### The bad news
$B$ is not a rotation matrix it is a more general linear transformation.  The bad news is that $B$ is singular when:

In [12]:
det(Binv) |> simplify

-cos(theta)

All Euler angle-based parameterizations have a singularity somewhere.  This is also refered to as *Gimbal Lock* and the physical interpretation is that at least two axis of a gimbal have become parallel.